<a href="https://colab.research.google.com/github/Kerriea-star/TensorFlow-Quickstart/blob/main/TensorFlow_2_Quickstart_for_Experts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import dependencies

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from keras.layers import Dense, Flatten, Conv2D
from keras import Model

TensorFlow version: 2.13.0


Load and prepare the MNIST dataset

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

11490434/11490434 [==============================] - 0s 0us/step


Use `tf.data` to batch and shuffle the dataset

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Build the `tf.keras` model using the Keras `model subclassing API`:

In [5]:
class MyModel(Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# create an instance of the model
model = MyModel()

choose an optimizer and loss function for training:

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

Select the metrics to measure the loss and accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [7]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

Use `tf.GradientTape` to train the model

In [8]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

Test the model:

In [9]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout)
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)


In [10]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
      f"Epoch {epoch + 1},"
      f'Loss: {train_loss.result()},'
      f'Accuracy: {train_accuracy.result() * 100},'
      f'Test Loss: {test_loss.result()},'
      f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1,Loss: 0.13411861658096313,Accuracy: 95.9949951171875,Test Loss: 0.06925918906927109,Test Accuracy: 97.80999755859375
Epoch 2,Loss: 0.041253626346588135,Accuracy: 98.70500183105469,Test Loss: 0.054596759378910065,Test Accuracy: 98.15999603271484
Epoch 3,Loss: 0.021367616951465607,Accuracy: 99.29666137695312,Test Loss: 0.04980674758553505,Test Accuracy: 98.38999938964844
Epoch 4,Loss: 0.012588081881403923,Accuracy: 99.58499908447266,Test Loss: 0.049963049590587616,Test Accuracy: 98.6199951171875
Epoch 5,Loss: 0.008774357847869396,Accuracy: 99.6933364868164,Test Loss: 0.07759326696395874,Test Accuracy: 98.06999969482422
